<a href="https://colab.research.google.com/github/riseofsanjay/python/blob/main/textsummarization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install --upgrade datasets

In [38]:
from datasets import load_dataset
ds = load_dataset("knkarthick/dialogsum")

In [39]:
!pip install --upgrade datasets

In [40]:
from datasets import load_dataset
ds = load_dataset("knkarthick/dialogsum")

In [41]:
ds


DatasetDict({
    train: Dataset({
        features: ['id', 'dialogue', 'summary', 'topic'],
        num_rows: 12460
    })
    validation: Dataset({
        features: ['id', 'dialogue', 'summary', 'topic'],
        num_rows: 500
    })
    test: Dataset({
        features: ['id', 'dialogue', 'summary', 'topic'],
        num_rows: 1500
    })
})

In [42]:
ds['train'][1]['dialogue']

"#Person1#: Hello Mrs. Parker, how have you been?\n#Person2#: Hello Dr. Peters. Just fine thank you. Ricky and I are here for his vaccines.\n#Person1#: Very well. Let's see, according to his vaccination record, Ricky has received his Polio, Tetanus and Hepatitis B shots. He is 14 months old, so he is due for Hepatitis A, Chickenpox and Measles shots.\n#Person2#: What about Rubella and Mumps?\n#Person1#: Well, I can only give him these for now, and after a couple of weeks I can administer the rest.\n#Person2#: OK, great. Doctor, I think I also may need a Tetanus booster. Last time I got it was maybe fifteen years ago!\n#Person1#: We will check our records and I'll have the nurse administer and the booster as well. Now, please hold Ricky's arm tight, this may sting a little."

In [43]:
ds['train'][1]['summary']

'Mrs Parker takes Ricky for his vaccines. Dr. Peters checks the record and then gives Ricky a vaccine.'

In [44]:
!pip install transformers


In [45]:
!pip install --upgrade transformers


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.8/10.8 MB 48.8 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.52.4
    Uninstalling transformers-4.52.4:
      Successfully uninstalled transformers-4.52.4


In [46]:
# Use a pipeline as a high-level helper
from transformers import pipeline

pipe = pipeline("summarization", model="facebook/bart-large-cnn")

Device set to use cpu


In [47]:
article_1 = ds['train'][1]['dialogue']

In [48]:
pipe(article_1, max_length=50, min_length=10, do_sample=False)

KeyboardInterrupt: 

In [ ]:
ds['train'][1]['summary']

In [ ]:
def preprocess_function(batch):
  source = batch['dialogue']
  target = batch['summary']
  source_ids = tokenizer(source, truncation=True, padding='max_length', max_length=128)
  target_ids = tokenizer(target, truncation=True, padding='max_length', max_length=128)

  labels = target_ids[' input_ids']
  labels =[[(label if label != tokenizer.pad_token_id else -100 ) for label in labels_examples] for labels_examples in labels]

  return{
      'input_ids': source_ids['input_ids'],
      'attention_mask': source_ids['attention_mask'],
      'labels': labels
  }


In [ ]:
df_source = ds.map(preprocess_function, batched=True)

In [ ]:
from transformers import BartTokenizer

tokenizer = BartTokenizer.from_pretrained("facebook/bart-large-cnn")

In [ ]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir='./content',
    per_device_train_batch_size=8,
    num_train_epochs=2,
    remove_unused_columns = True,

)

In [ ]:
trainer = Trainer(
    model = model,
    args=training_args,
    train_dataset=df_source[' train'],
    eval_dataset=df_source['test']
)

In [ ]:
trainer.train()